In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

url_base = 'https://www.zapimoveis.com.br/venda/?transacao=venda&pagina='
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'
}

# Listas para armazenar as informações coletadas
locations = []
adress = []
descriptionHouse = []
tamanho_imoveis = []
Rooms = []
Bathroom = []
garage = []
links = []
valueHs = []

# Loop para percorrer as páginas disponíveis
for i in range(1, 101):
    url = url_base + str(i)
    site = requests.get(url, headers=headers)

    # Verifica se a página retornou um resultado válido (código 200)
    if site.status_code == 200:
        soup = BeautifulSoup(site.content, 'html.parser')

        # Obtendo os anúncios
        for name in soup.find_all('h2', class_='card__address'):
            locations.append(name.get_text())

        # Pegando o endereço
        for adr in soup.find_all('p', class_='card__street'):
            adress.append(adr.get_text())

        # Pegando a descrição do imóvel
        for descr in soup.find_all('p', class_='card__description'):
            descriptionHouse.append(descr.get_text())

        # Secção onde está localizado as informações
        secao_amenities = soup.find_all('section', class_='card__amenities')

        # Tamanho do imóvel
        for secao in secao_amenities:
            elem_p_list = secao.find_all('p', class_='card__amenity', itemprop='floorSize')
            if elem_p_list:
                tamanho_imovel = elem_p_list[-1].get_text(strip=True)
                tamanho_imoveis.append(tamanho_imovel)
            else:
                tamanho_imoveis.append("Informação não encontrada")

        # Número de quartos
        for rom in secao_amenities:
            elem_p_list = rom.find_all('p', class_='card__amenity', itemprop='numberOfRooms')
            if elem_p_list:
                numberRooms = elem_p_list[-1].get_text(strip=True)
                Rooms.append(numberRooms)
            else:
                Rooms.append("Informação não encontrada")

        # Número de banheiros
        for bath in secao_amenities:
            elem_p_list = bath.find_all('p', class_='card__amenity', itemprop='numberOfBathroomsTotal')
            if elem_p_list:
                numberBathrooms = elem_p_list[-1].get_text(strip=True)
                Bathroom.append(numberBathrooms)
            else:
                Bathroom.append("Informação não encontrada")

        # Informações sobre vaga garagem
        for gar in secao_amenities:
            elem_p_list = gar.find_all('p', class_='card__amenity')
            if elem_p_list:
                numberGarage = elem_p_list[-1].get_text(strip=True)
                garage.append(numberGarage)
            else:
                garage.append("Informação não encontrada")

        # Pegando informações de valores
        for value in soup.find_all('div', class_='listing-price'):
            valueHs.append(value.get_text(separator=' '))

        # Pegando o link da postagem
        for link in soup.find_all('div', class_='result-card'):
            links.append(link.find('a')['href'])
    else:
        # Caso a página não exista (erro 404), interrompe o loop
        print(f"Erro ao acessar a página {url}. Status code: {site.status_code}")
        break

# Criando DataFrame
df = pd.DataFrame({'Localização': locations,
                   'Endereço': adress,
                   'Descrição': descriptionHouse,
                   'Tamanho': tamanho_imoveis,
                   'Quartos': Rooms,
                   'Banheiros': Bathroom,
                   'Vagas Garagem': garage,
                   'Valores': valueHs,
                   'Links': links
                   })

df
